In [11]:
# --- Install Dependencies ---
!pip install -q paramiko python-dotenv ipywidgets

In [12]:
# --- Imports ---
import os
import subprocess
import json
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display
import paramiko

# --- Load Environment Variables ---
load_dotenv()
server = os.getenv("server")
username = os.getenv("username")
key_path = os.getenv("key")  # Path to SSH key
remote_host = f"{username}@{server}"
remote_path = "/home/codechallenge/InnoClass/backend/classEngine/data/"

# --- Check Available Files ---
files = [f for f in os.listdir('.') if f.endswith('.sen.gz')]
files2 = [f for f in os.listdir('.') if f.endswith('.dat')]

if not files:
    print(" No `.sen.gz` files found in this directory. Please extract patent sentences.")
if not files2:
    print(" No `.dat` SDG target files found. Please run `extract_sdg_text.ipynb` first.")

# --- Widgets ---
file_selector = widgets.Select(
    options=files,
    description=' Patents:',
    rows=10,
    layout=widgets.Layout(width='250px')
)

file_selector2 = widgets.Select(
    options=files2,
    description=' SDG Targets:',
    rows=10,
    layout=widgets.Layout(width='250px')
)

text1 = widgets.Text(value="Target Text", description=" Prompt:")
text2 = widgets.Text(value="sentence", description=" Patent Field:")

send_button = widgets.Button(description=" Send to Server", button_style='primary')
execute_button = widgets.Button(description=" Execute Search", button_style='success')

# --- Transfer Files Function ---
def send_data_to_server():
    filename = file_selector.value
    filename2 = file_selector2.value
    filename3 = "searchconfig.json"

    # Create JSON config
    config = {
        "filename_sdg": filename2,
        "filename_texts": filename,
        "Prompt": text1.value,
        "extracted_text": text2.value
    }

    with open(filename3, "w") as f:
        json.dump(config, f, indent=4)

    # SCP Commands
    commands = [
        ["scp", filename, f"{remote_host}:{remote_path}"],
        ["scp", filename2, f"{remote_host}:{remote_path}"],
        ["scp", filename3, f"{remote_host}:{remote_path}"]
    ]

    for i, cmd in enumerate(commands, 1):
        print(f" Command {i}: {' '.join(cmd)}")
        try:
            proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            stdout, stderr = proc.communicate()
            if proc.returncode == 0:
                print(f" Successfully transferred {cmd[1].split(':')[0]}")
            else:
                print(f" Error transferring file: {stderr}")
        except Exception as e:
            print(f" Unexpected error: {e}")

    print(" All files processed.\n")

# --- Execute Remote Docker Search ---
def execute_remote_command(hostname, username, private_key_path, command):
    try:
        key = paramiko.Ed25519Key.from_private_key_file(private_key_path)
        client = paramiko.SSHClient()
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        client.connect(hostname, username=username, pkey=key)
        stdin, stdout, stderr = client.exec_command(command)
        output = stdout.read().decode()
        error = stderr.read().decode()
        client.close()

        if error:
            print("Error during execution:\n", error)
        else:
            print("Command output:\n", output)
    except Exception as e:
        print(f"❗ SSH Error: {e}")

# --- On-click handlers ---
def on_send_click(_):
    send_data_to_server()

def on_execute_click(_):
    command = "cd /home/codechallenge/InnoClass/backend/ && docker-compose up embedsearch"
    execute_remote_command(server, username, key_path, command)

# --- Bind Buttons ---
send_button.on_click(on_send_click)
execute_button.on_click(on_execute_click)

# --- Display Widgets ---
display(send_button, execute_button)
display(file_selector, file_selector2, text1, text2)

 No `.sen.gz` files found in this directory. Please extract patent sentences.


Button(button_style='primary', description=' Send to Server', style=ButtonStyle())

Button(button_style='success', description=' Execute Search', style=ButtonStyle())

Select(description=' Patents:', layout=Layout(width='250px'), options=(), rows=10, value=None)

Select(description=' SDG Targets:', layout=Layout(width='250px'), options=('sgd_goals.dat', 'sgd_targets.dat')…

Text(value='Target Text', description=' Prompt:')

Text(value='sentence', description=' Patent Field:')